# Методы, специфичные для трансформеров

В трансформерах есть механизм **self-attention**, который кажется естественным способом определить, какие токены текста/кусочки изображения имеют большую важность для предсказания.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.1/L14/self_attention.png"  width="900"></center>

Это бы отлично работало, будь у нас только один блок **self-attention**, но архитектура трансформера состоит из **нескольких блоков кодера/декодера**, расположенных друг за другом. От слоя к слою за счет того же **self-attention** **информация в эмбеддингах перемешивается** сильнее и сильнее. Так, например, для классификации текста мы можем использовать эмбеддинг на выходе **BERT**, соответствующий `[CLS]` токену, в котором на входе **BERT** не было никакой информации о последующем тексте.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.1/L14/transformer_text_translation_example.png" width="800"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-bert/">The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)</a></em></center>

Кроме того, в стандартном блоке трансформера есть **residual соединения**. Из-за этого информация о токенах/патчах проходит не только через **attention**, но и через **residual соединения**.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L14/out/transformer_architecture.png" width="450"></center>

<center><em>Архитектура трансформера</em></center>

<center><em>Source: <a href="https://arxiv.org/pdf/1706.03762.pdf"> Attention Is All You Need</a></em></center>

По этим причинам объяснение работы трансформера через attention — непростая задача.

**Предупреждение:** методы объяснения **attention** только частично объясняют работу трансформера, они  разнообразны и могут давать противоречивые результаты.

Давайте для начала немного поизучаем, как выглядят значения self-attention в трансформерах. Подгружаем библиотеку.

In [ ]:
!pip install -q transformers[sentencepiece]

Возьмем базовый русскоязычный разговорный [BERT от Deep Pavlov 🛠️[doc]](https://huggingface.co/DeepPavlov/rubert-base-cased-conversational), [обученный 🛠️[doc]](https://huggingface.co/blanchefort/rubert-base-cased-sentiment) для определения эмоциональной окраски коротких русских текстов. Загружаем токенизатор и модель. Ставим флаг `output_attentions=True`, чтобы модель возвращала значения attention.

In [ ]:
import transformers
from transformers import BertTokenizerFast, AutoModelForSequenceClassification
from IPython.display import clear_output

tokenizer = BertTokenizerFast.from_pretrained(
    "blanchefort/rubert-base-cased-sentiment",
)
model = AutoModelForSequenceClassification.from_pretrained(
    "blanchefort/rubert-base-cased-sentiment",
    output_attentions=True,  # for save attention
)
clear_output()

Готовим предложения.

In [ ]:
sentences = [
    "Мама мыла раму",
    "Фильм сделан откровенно плохо",
    "Максимально скучный сериал, где сюжет высосан из пальца",
    "Я был в восторге",
    "В общем, кино хорошее и есть много что пообсуждать",
]

tokens = [
    ["[cls]"] + tokenizer.tokenize(sentence) + ["[sep]"] for sentence in sentences
]

Посмотрим, как разбивается на токены предложение. На выходе токенизатора номера токенов.

In [ ]:
item = 0
print(f"Tokens: {tokens[item]}")
token_ids = [tokenizer.encode(sentence) for sentence in sentences]
print(f"Token ids: {token_ids[item]}")

Посмотрим на предсказания модели, чтобы проверить, насколько она адекватна.

In [ ]:
import torch


ans = {0: "NEUTRAL", 1: "POSITIVE", 2: "NEGATIVE"}

for item in range(5):
    input_ids = torch.tensor([token_ids[item]])
    model_output = model(input_ids)
    predicted = torch.argmax(model_output.logits, dim=1).numpy()
    print(f"Text: {sentences[item]}")
    print(f"Predict lable = {predicted}, {ans[predicted.item()]}")

В данной модели 12 слоев (блоков трансформеров), поэтому модель возвращает кортеж из 12 тензоров. Каждый слой имеет 12 голов self-attention.

In [ ]:
item = 1
input_ids = torch.tensor([token_ids[item]])
model_output = model(input_ids)

attentions = model_output.attentions
print(f"Text: {sentences[item]}")
print(f"Tokens: {tokens[item]}")
print(f"Number of layers: {len(attentions)}")
print(
    f"Attention size: {attentions[0].shape} "
    "[batch x attention_heads x seq_size x seq_size]"
)

Преобразуем в однородный массив для удобства манипуляций.

Код этой части лекции основана на:
* [[git] 🐾 Attention flow](https://github.com/samiraabnar/attention_flow),
* [[arxiv] 🎓 Quantifying Attention Flow in Transformers (Abnar, Zuidema, 2020)](https://arxiv.org/pdf/2005.00928.pdf).


In [ ]:
import numpy as np


def to_array(attentions):
    attentions_arr = [attention.detach().numpy() for attention in attentions]
    return np.asarray(attentions_arr)[:, 0]


attentions_arr = to_array(attentions)
print(
    f"Shape: {attentions_arr.shape} " "[layers x attention_heads x seq_size x seq_size]"
)
print(f"Type: {type(attentions_arr)}, Dtype: {attentions_arr.dtype}")

Посмотрим на **нулевую голову нулевого слоя**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

x_ticks = tokens[item]
y_ticks = tokens[item]

sns.heatmap(
    data=attentions_arr[0][0],
    vmin=0,
    vmax=1,
    xticklabels=x_ticks,
    yticklabels=y_ticks,
    cmap="YlOrRd",
)

plt.show()

Тут по **оси x** — токены, **на** которые смотрит внимание, по **оси y** — токены, **куда** записывается результат прохождения слоя.

Так для слова “плохо” первая голова внимания первого слоя больше всего смотрит на слова “фильм” и “ откровенно”.

Для визуализации внимания можно использовать библиотеку [BertViz 🛠️[doc]](https://pypi.org/project/bertviz/):
* [[arxiv] 🎓 Visualizing Attention in Transformer-Based Language Representation Models (Vig, 2019)](https://arxiv.org/pdf/1904.02679.pdf),
* [[colab] 🥨 BertViz Interactive Tutorial](https://colab.research.google.com/drive/1hXIQ77A4TYS4y3UthWF-Ci7V7vVUoxmQ?usp=sharing).

In [ ]:
!pip install -q bertviz

Тут Layer — это выбор слоя, цвета — головы self-attention, слева — куда записывается, справа — на какие токены смотрит. Яркость соединяющих линий — величина attention (чем ярче, тем больше). Картину, аналогичную картине выше, можно получить, *дважды щелкнув на первый синий квадрат*.

In [ ]:
import bertviz
from bertviz import head_view

head_view(model_output.attentions, tokens[item])

Усредним значения по головам:

In [ ]:
attention_head_mean = attentions_arr.mean(axis=1)
print(f"{attention_head_mean.shape} [layers x seq_size x seq_size]")

Посмотрим на усредненное по головам внимание **на первом слое**:

In [ ]:
x_ticks = tokens[item]
y_ticks = tokens[item]

sns.heatmap(
    data=attention_head_mean[0],
    vmin=0,
    vmax=1,
    xticklabels=x_ticks,
    yticklabels=y_ticks,
    cmap="YlOrRd",
)

plt.show()

И на **последнем слое**:

In [ ]:
x_ticks = tokens[item]
y_ticks = tokens[item]

sns.heatmap(
    data=attention_head_mean[-1],
    vmin=0,
    vmax=1,
    xticklabels=x_ticks,
    yticklabels=y_ticks,
    cmap="YlOrRd",
)

plt.show()

Видно, что на последнем слое внимание распределено между токенами практически равномерно.

Посмотрим на **значения внимания для записываемого в эмбеддинг [CLS] токена** (эмбеддинг с него используется для классификации).

In [ ]:
x_ticks = tokens[item][1:-1]
y_ticks = [i for i in range(12, 0, -1)]

sns.heatmap(
    data=np.flip(attention_head_mean[:, 0, 1:-1], axis=0),
    xticklabels=x_ticks,
    yticklabels=y_ticks,
    cmap="YlOrRd",
)

plt.show()

Видно, что после 6-го слоя значения внимания выравниваются. Это связано с тем, что механизм **self-attention** смешивает информацию о токенах.

<font size="5">Residual connection</font>

Давайте сначала определимся, что делать с **residual соединениями**.

**Residual соединение** можно записать как

$$ \large V_{l+1} = V_l + W_{att}V_l,$$

где $W_{att}$ — матрица внимания, а $V_l$ — эмбеддинги.

После нормализации это можно переписать как

$$\large A=0.5W_{att}+0.5I,$$

где $I$ — единичная матрица.

[[arxiv] 🎓 Quantifying Attention Flow in Transformers (Abnar, Zuidema, 2020)](https://arxiv.org/pdf/2005.00928.pdf)

In [ ]:
def residual(attention_head_mean):
    attention_residual = (
        0.5 * attention_head_mean
        + 0.5 * np.eye(attention_head_mean.shape[1])[None, ...]
    )
    return attention_residual


attention_res = residual(attention_head_mean)

x_ticks = tokens[item][1:-1]
y_ticks = [i for i in range(12, 0, -1)]

sns.heatmap(
    data=np.flip(attention_res[:, 0, 1:-1], axis=0),
    xticklabels=x_ticks,
    yticklabels=y_ticks,
    cmap="YlOrRd",
)

plt.show()

## Attention rollout
“Разворачивание внимания” (**Attention rollout**) — предложенный в [статье 🎓[arxiv]](https://arxiv.org/pdf/2005.00928.pdf) способ отслеживания информации, распространяемой от входного к выходному блоку, в котором значение внимания рассматривается как доля пропускаемой информации.  Доли информации перемножаются и суммируются. Итоговая формула — рекурсивное матричное перемножение.

\begin{align}
\widetilde{A}(l_i) = \left\{
\begin{array}{cl}
A(l_i)\widetilde{A}(l_{i-1}) & i>0 \\
A(l_i) & i = 0.
\end{array}
\right.
\end{align}



In [ ]:
def rollout(attention_res):
    rollout_attention = np.zeros(attention_res.shape)
    rollout_attention[0] = attention_res[0]
    n_layers = attention_res.shape[0]
    for i in range(1, n_layers):
        rollout_attention[i] = attention_res[i].dot(rollout_attention[i - 1])
    return rollout_attention

In [ ]:
rollout_attention = rollout(attention_res)

In [ ]:
x_ticks = tokens[item][1:-1]
y_ticks = [i for i in range(12, 0, -1)]

sns.heatmap(
    data=np.flip(rollout_attention[:, 0, 1:-1], axis=0),
    xticklabels=x_ticks,
    yticklabels=y_ticks,
    cmap="YlOrRd",
)

plt.show()

Реализации от HuggingFace:
* [[git] 🐾 Для ViT и картинок](https://huggingface.co/spaces/probing-vits/attention-rollout/tree/main)
* [[git] 🐾 На PyTorch Rollout для BERT и некоторых других методов](https://huggingface.co/spaces/amsterdamNLP/attention-rollout/tree/main)
* [[demo] 🎮 Attention Rollout — RoBERTa](https://huggingface.co/spaces/amsterdamNLP/attention-rollout)

## Attention Flow

Другим вариантом рассмотрения распространения внимания является **attention flow**. В нем трансформер представляют в виде направленного графа, **узлы** которого представляют собой **эмбеддинги** между слоями, а **ребра** — связи в виде **attention** с ограниченной емкостью (передающей способностью).

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.1/L14/attention_flow.png" width="700">

<em>Source: <a href="https://github.com/samiraabnar/attention_flow/blob/master/bert_example.ipynb">Bert Example
</a></em></center>

В такой постановке задача нахождения роли токенов/частей изображения в результирующем эмбеддинге сводится к [задаче о максимальном потоке 📚[wiki]](https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_%D0%BE_%D0%BC%D0%B0%D0%BA%D1%81%D0%B8%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%BC_%D0%BF%D0%BE%D1%82%D0%BE%D0%BA%D0%B5) (задача нахождения такого потока по транспортной сети, что сумма потоков в пункт назначения была максимальна). Это известная алгоритмическая задача, которую мы не будем разбирать в рамках этого курса.

* [[git] 🐾 Attention flow](https://github.com/samiraabnar/attention_flow)
* [[arxiv] 🎓 Quantifying Attention Flow in Transformers (Abnar, Zuidema, 2020)](https://arxiv.org/pdf/2005.00928.pdf)



## Gradient-weighted Attention Rollout

Метод, объединяющий GradCam и Attention Rollout, позволяет оценить положительный и отрицательный вклад токенов / частей изображения в итоговый результат.
* [[article] 🎓 Transformer Interpretability Beyond Attention Visualization (Chefer et.al., 2021)](https://openaccess.thecvf.com/content/CVPR2021/papers/Chefer_Transformer_Interpretability_Beyond_Attention_Visualization_CVPR_2021_paper.pdf)
* [[git] 🐾 Attention Rollout](https://huggingface.co/spaces/amsterdamNLP/attention-rollout/tree/main)
* [[demo] 🎮 Attention Rollout — RoBERTa](https://huggingface.co/spaces/amsterdamNLP/attention-rollout)

Методы, основанные на механизме внимания, только частично объясняют работу трансформера и на сегодняшний день не являются надежным методом оценки важности признаков.